# Functional Test 6.1.4 - Flash XDMA shell with FireSim code into an FPGA persistent flash or RAM and validate

This Jupyter notebook will allow you to flash experimenter FPGA code based on [XDMA shell](https://docs.fires.im/en/stable/Getting-Started-Guides/On-Premises-FPGA-Getting-Started/Initial-Setup/Xilinx-Alveo-U280.html#install-your-fpga-s) into the FPGA persistent flash or RAM. If the persistent flash is used, the end result is an FPGA that even after a cold reboot of the server retains its programming with a standard [Xilinx XDMA shell](https://github.com/Xilinx/dma_ip_drivers/tree/master/XDMA). If the program is flashed into RAM, a warm reboot of the server will activate it. 

This procedure can be used to reset the FPGA at a given site after experiments or initialize a newly installed device. 

It is assumed you are operating as part of the FABRIC Maintenance project and have access to the persistent volume named `fpga-tools` created on EDC where releavent tools are downloaded.

This notebook is broken up into steps and does not have to be executed in sequence.

- Step 2 Has multiple cells that initialize fablib and variables and creates a new slice
  - The cells initializing the state __must always be executed__.
  - Creating a slice is optional and can be skipped if slice already exists
- Step 3 is a re-entrant cell and can be used any time you want to refresh information about the existing slice
- Step 4 fetches the tools from the Storage VM and installs:
  - Xilinx Labtools
  - experimenter-provided bitfile artifact
- Step 5 is the actual flashing and it involves identifying and rebooting the underlying server.
- Step 6 is an optional 'extend the slice' step that can be excuted if Steps 2 and 3 have been executed.
- Step 7 is 'delete the slice' that can be executed any time after Steps 2 and 3 have been executed.
 

## Step 0: Re-create a VM attached to fpga-tools volume on EDC

In order to have access to necessary tools execute the notebook to [re-create a Storage VM attached](../../fablib_api/fabric_fpgas/fpga_tools_storage.ipynb) to the `fpga-tools` persistent storage. You must execute it as a member of FABRIC Staff project. 

## Step 1: Identify and isolate the worker node

Unless the whole site is already in maintenance, using administrator tools identify the worker node with FPGA and put it in maintenance making sure it does not have experimenter VMs on it. You can check the [aggregate ads in JSON](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON) to make sure you are targeting the right worker.

## Step 2: Provision a VM on the desired worker with attached FPGA and FABNetv4 connection

Create a slice with a VM attached to the FPGA on the desired site and a FABNetv4 interface to reach the Storage VM in Step 0.

### Initialize fablib and variables

In [ ]:
# Initialize FABlib

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Define slice parameters - re-execute as needed to run any of the steps in this notebook.

In [ ]:
# user artifact should be deposited Storage VM into /mnt/fpga_tools/static/artifacts/<owner username>/<version> since names of artifacts may be similar or same.
artifact_owner_username = 'xdma_firesim'
artifact_version = 'v1'

# edit the name of the user-provided artifact and labtools and DPDK docker images stored in Storage VM as needed

artifact = 'firesim.mcs'
artifact_golden = 'revert_to_golden.mcs'

# Xilinx labtools package in Storage VM
labtools_package = 'Vivado/Vivado_Lab_Lin_2023.2_1013_2256.tar.gz'

# Xilinx xrt package in Storage VM
xrt_package = 'alveo-packages/xrt_202320.2.16.204_20.04-amd64-xrt.deb'

# Xilinx deployment target package in Storage VM
deployment_tgt_package = 'alveo-packages/xilinx-u280-gen3x16-xdma_2023.2_2023_1014_0238-all.deb.tar.gz'

# Xilinx xbflash2 package in Storage VM
xbflash2_package = 'alveo-packages/xrt_202210.2.13.466_20.04-amd64-xbflash2.deb'

# Pcimem configDPDK.sh script in Storage VM
configdpdk_script = 'alveo-utilities/configDPDK.sh'

# FABNetv4 of storage VM - consult the Storage VM slice for this FABNetv4 IP address
storage_vm_ip = "10.132.129.2"
# username and password used in storage VM
nginx_user = "fpga_tools"
nginx_password = "secret-password"

#
# should not need to edit below
FPGA_CHOICE='FPGA_Xilinx_U280'


# don't edit - convert from FPGA type to a resource column name
# to use in filter lambda function below
choice_to_column = {
    "FPGA_Xilinx_U280": "fpga_u280_available",
}

column_name = choice_to_column.get(FPGA_CHOICE, "Unknown")

fpga_bdf = "0000:1f:00.0"

#fablib.get_image_names()

In [ ]:
# setup site name
site='RENC'
node_name='fpga-node'

# name the slice and the node 
slice_name=f'Persistent slice with {FPGA_CHOICE} on {site}'
print(f'Will create slice "{slice_name}"')

### Create a new slice 

Create a slice with FPGA component on selected site and access to FABNetv4 network.

__NOTE:__ It is important to use a Docker-enabled image so that Docker can properly build docker images on IPv6-enabled sites.

In [ ]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)
image = 'docker_ubuntu_20'

# Add node with a 200G drive and 8 of CPU cores using Ubuntu 20 image
node = slice.add_node(name=node_name, site=site, cores=8, disk=200, image=image)
node.add_component(model=FPGA_CHOICE, name='fpga1')
# be sure to add FABNetv4 so we can communicate with the slice that has the tools
node.add_fabnet()

# use the postboot script from docker examples
node.add_post_boot_upload_directory('../../fablib_api/docker_containers/node_tools','.')
node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
node.add_post_boot_upload_directory('node_config','.')
node.add_post_boot_execute(f'chmod a+x node_config/ipv6-and-docker-plugins.sh && node_config/ipv6-and-docker-plugins.sh')

# Submit Slice Request
slice.submit();

Add storage VM into /etc/hosts for convenience. __Consult the storage slice for the FABNetv4 IPv4 address of that VM.__

In [ ]:
slice = fablib.get_slice(slice_name)
node = slice.get_node(name=node_name)   

commands = list()
commands.append(f"echo {storage_vm_ip} fpga-tools-host | sudo tee -a /etc/hosts")
commands.append(f"echo 127.0.0.1 {node_name} | sudo tee -a /etc/hosts")

for command in commands:
    stdout, stderr = node.execute(command)

## Step 3: Inspect the slice
Note that nat64 configuration is done at boot time.

In [ ]:
slice = fablib.get_slice(slice_name)

node = slice.get_node(name=node_name)              

node_addr = node.get_interface(network_name=f'FABNET_IPv4_{node.get_site()}').get_ip_addr()

slice.show()
slice.list_nodes()
slice.list_networks()
print(f'Node FABNetV4 IP Address is {node_addr}')

## Step 4: Fetch Tools and Install Vivado Lab



### 4.1 Fetch the Linux tools from Xilinx and place into appropriate location

#### 4.1.1 Clone OCT-FPGA repo

In [ ]:
# checkout the repo

#
# This repo is forked from https://github.com/OCT-FPGA/P4OpenNIC_Public.git
# program_flash.tcl is modified to include commands for booting from the 
# configuration memory device
#

tools_neu_repo = 'https://github.com/mcevik0/P4OpenNIC_Public.git'

commands = list()
commands.append(f'[ ! -d ~/fpga-tools ] && mkdir -p ~/fpga-tools')
commands.append(f'cd fpga-tools && git clone {tools_neu_repo} && cd P4OpenNIC_Public && git checkout fpga-flash-boot-from-config-mem')

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

#### 4.1.2 Clone pcimem repo

In [ ]:
# checkout the repo

# checkout the repo
pcimem_repo = 'https://github.com/billfarrow/pcimem.git'

commands = list()
commands.append(f'[ ! -d ~/fpga-tools ] && mkdir -p fpga-tools')
commands.append(f'cd fpga-tools && git clone {pcimem_repo}')

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

#### 4.1.3 Copy configDPDK.sh from the Storage VM

In [ ]:
import os.path

tools_location = '~/fpga-tools'

commands = list()
commands.append(f'[ ! -d {tools_location} ] && mkdir -p {tools_location}')
commands.append(f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{configdpdk_script}  > {tools_location}/{os.path.basename(configdpdk_script)}')
commands.append(f'chmod +x {tools_location}/{os.path.basename(configdpdk_script)}')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.1.4 Clone open-nic-driver repo

In [ ]:
# checkout the repo - open-nic-driver
opennicdriver_repo = 'https://github.com/Xilinx/open-nic-driver.git'

commands = list()
commands.append(f'[ ! -d ~/fpga-tools ] && mkdir -p fpga-tools')
commands.append(f'cd fpga-tools && git clone {opennicdriver_repo}')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.1.5 Next transfer the Xilinx Vivado Lab package from the Storage VM

In [ ]:
import os.path

tools_location = '~/xilinx-labtools/vivado-installer'

commands = [f'[ ! -d {tools_location} ] && mkdir -p {tools_location}',
            f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{labtools_package}  > {tools_location}/{os.path.basename(labtools_package)}']

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.1.6 Next transfer the artifact from Storage VM into a folder in this repo. 
This file will be called `<application_name>.mcs` and should be placed in the `~/bitfile` directory before starting the firmware build.

In [ ]:
bitfile_location = '~/fpga-bitfile'

commands = [f'[ ! -d {bitfile_location} ] && mkdir -p {bitfile_location}',
            f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/artifacts/{artifact_owner_username}/{artifact_version}/{artifact}  > {bitfile_location}/{artifact}',
            f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/xrt/{artifact_golden}  > {bitfile_location}/{artifact_golden}']


for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

### 4.2 Install - Vivado Lab Tools

#### 4.2.1 Extract and install Xilinx Vivado Lab package

In [ ]:
# parse package name to get version
import re

pattern = re.compile('(Xilinx_)?Vivado_Lab_Lin_([\d]{4}.[\d]_[\d]{4}_[\d]{4}).tar.gz')
m = pattern.match(os.path.basename(labtools_package))
if m:
    version = m[2]
else:
    version = 'Unknown'


vivado_install_dir = "/tools/Xilinx"
 
commands = list()
commands.append(f'tar xf {tools_location}/{os.path.basename(labtools_package)} -C {tools_location}')
commands.append(f'sudo {tools_location}/Vivado_Lab_Lin_{version}/xsetup --agree 3rdPartyEULA,XilinxEULA --batch Install --edition "Vivado Lab Edition (Standalone)" --location {vivado_install_dir}')


for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.2.2 Install necessary libs

In [ ]:
commands = list()
commands.append(f'sudo apt install -y lsb &> /tmp/fpga-apt-install.out')
commands.append(f'sudo {tools_location}/Vivado_Lab_Lin_{version}/installLibs.sh &> /tmp/fpga-installLibs.out')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.2.3 Install JTAG drivers

In [ ]:
import re

pattern = re.compile('([\d]{4}.[\d])_([\d]{4}_[\d]{4})')
m = pattern.match(os.path.basename(version))
if m:
    version_major = m[1]
else:
    version_major = 'Unknown'


commands = list()
commands.append(f'sudo {vivado_install_dir}/Vivado_Lab/{version_major}/data/xicom/cable_drivers/lin64/install_script/install_drivers/install_drivers')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.2.4 Update packages

In [ ]:
commands = list()
commands.append(f'(sudo apt -y update && sudo apt -y upgrade) &> /tmp/fpga-apt-upgrade.log')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.2.5 Reboot

In [ ]:
reboot = 'sudo reboot'
try:
    print(reboot)
    node.execute(reboot)
    
    slice.wait_ssh(timeout=360,interval=10,progress=True)

    print("Now testing SSH abilites to reconnect...",end="")
    slice.update()
    slice.test_ssh()
    print("Reconnected!")

except Exception as e:
    print(f"Fail: {e}")  

node.config()
print("Done")    

#### 4.2.6 Get the JTAG ID


In [ ]:
commands = list()
commands.append(f'source {vivado_install_dir}/Vivado_Lab/{version_major}/settings64.sh && vivado_lab -mode batch -source ~/fpga-tools/P4OpenNIC_Public/FABRIC-P4/scripts/get_jtag.tcl | grep -o "Xilinx/[^[:space:]]*" | cut -d/ -f2 > /tmp/fpga-jtag.id')
commands.append(f'echo {fpga_bdf} > /tmp/fpga-bdf.id')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.2.7 Install kernel-headers

In [ ]:
commands = list()
commands.append(f'sudo apt install -y linux-headers-$(uname -r) &> /tmp/fpga-apt-kernel-headers.log')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

#### 4.2.8 Install libraries for running Vivado GUI

In [ ]:
commands = list()
commands.append(f'sudo apt-get install -y libxrender1 libxtst6 libxi6 &> /tmp/fpga-apt-libs.log')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

### 4.3. Install - Alveo Packages
https://www.xilinx.com/support/download/index.html/content/xilinx/en/downloadNav/alveo/u280.html

1. Xilinx Runtime

   The Xilinx runtime (XRT) is a low level communication layer (APIs and drivers) between the host and the card.
   - [xrt_202320.2.16.204_20.04-amd64-xrt.deb](https://www.xilinx.com/bin/public/openDownload?filename=xrt_202320.2.16.204_20.04-amd64-xrt.deb)

 
2. Deployment Target Platform

   The deployment target platform is the communication layer physically implemented and flashed into the card.
   - [xilinx-u280-gen3x16-xdma_2023.2_2023_1014_0238-all.deb.tar.gz](https://www.xilinx.com/bin/public/openDownload?filename=xilinx-u280-gen3x16-xdma_2023.2_2023_1014_0238-all.deb.tar.gz)


In [ ]:
# This scheme is taking long to download the packages from the repos.
# Use the following scheme

###commands = list()
###commands.append(f'sudo wget -qO - https://www.xilinx.com/support/download/2020-2/xilinx-master-signing-key.asc | sudo apt-key add -')
###commands.append(f'echo "deb https://packages.xilinx.com/artifactory/debian-packages $(lsb_release -cs) main" | sudo tee -a /etc/apt/sources.list.d/xlnx.list')
###commands.append(f'(sudo apt-get update && sudo apt-get install -y xilinx-u280-gen3x16-xdma-base=1-3585717 xilinx-u280-gen3x16-xdma-validate=1-3585755 xilinx-cmc-u280=1.3.5-3592445 xilinx-sc-fw-u280=4.3.28-1.ea1b92f ) &> /tmp/fpga-apt-alveo.log')

###for command in commands:
###   print(f'--- Node {node_name}: Executing command: {command}')
###   stdout, stderr = node.execute(command)
###print('--- Done')

xbflash2 and Xilinx Runtime Target Platform - Download

In [ ]:
import os.path

tools_location = '~/xilinx-labtools/alveo-packages'

commands = list()
commands.append(f'[ ! -d {tools_location} ] && mkdir -p {tools_location}')
###commands.append(f'wget -q https://www.xilinx.com/bin/public/openDownload?filename=xrt_202210.2.13.466_20.04-amd64-xbflash2.deb -O {tools_location}/xrt_202210.2.13.466_20.04-amd64-xbflash2.deb')
commands.append(f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{xbflash2_package}  > {tools_location}/{os.path.basename(xbflash2_package)}')
commands.append(f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{xrt_package}  > {tools_location}/{os.path.basename(xrt_package)}')
commands.append(f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{deployment_tgt_package}  > {tools_location}/{os.path.basename(deployment_tgt_package)}')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

xbflash2 and Xilinx Runtime Target Platform - Install

In [ ]:
import os.path

tools_location = '~/xilinx-labtools/alveo-packages'

commands = list()
commands.append(f'[ ! -d {tools_location} ] && mkdir -p {tools_location}')

commands.append(f'sudo apt-get install -y {tools_location}/xrt_202210.2.13.466_20.04-amd64-xbflash2.deb &> /tmp/fpga-apt-xbflash2.log')
commands.append(f'sudo mkdir {tools_location}/xrt_platform')
commands.append(f'sudo tar -zxvf {tools_location}/{os.path.basename(deployment_tgt_package)} -C {tools_location}/xrt_platform')
commands.append(f'cd {tools_location}/xrt_platform && sudo apt-get install -y ./*.deb &> /tmp/fpga-apt-alveo.log')


for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

## Step 5: Flash the card 

This section includes 2 methods using Vivado Labtools - via CLI and GUI. Also a third method with xbflash2 is added.

Use Method-1 or Method-2 or Method-3.

### Method 1 - Flash the card via CLI

Summary:
- program_flash.tcl is customized to boot the FPGA from the configuration memory device as the last step. 
- Cold-rebooting of the server is not needed. 
- On the server, PCI bus is rescanned - see https://fabric-testbed.atlassian.net/browse/FIP-1528

Execute the block below

In [ ]:
commands = list()
commands.append(f'source {vivado_install_dir}/Vivado_Lab/{version_major}/settings64.sh && export JTAG_ID=`cat /tmp/fpga-jtag.id` && export EXTENDED_DEVICE_BDF1=`cat /tmp/fpga-bdf.id` && cd ~/fpga-tools/P4OpenNIC_Public/FABRIC-P4/scripts && pwd && echo $JTAG_ID && ./program_flash.sh {bitfile_location}/{artifact} au280 $JTAG_ID')


for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)

print('Done')    

Execute PCIbus rescan on the worker node

```
/opt/mlnx-scripts/pcie_disable_fatal.sh 25:00.0
/opt/mlnx-scripts//pcie_disable_fatal.sh 25:00.1
echo 1 > /sys/bus/pci/devices/0000\:25\:00.0/remove
echo 1 > /sys/bus/pci/devices/0000\:25\:00.1/remove
echo 1 > /sys/bus/pci/rescan
```

#### <font color='red'>STOP - This is the end of flashing with Method-1</font> 

### Method 2 - Flash the card Use Vivado Hardware Manager and boot from configuration memory device



#### 5.2.1 - Open Vivado

Login to the VM
```
ssh -X -F ~/.ssh/<ssh_client_config> -i ~/.ssh/<sliver_key> ubuntu@<IP-address>
```

On the VM, execute the following to open Vivado GUI:
```
export VitisNetP4_Option_VISIBLE=true
source /tools/Xilinx/Vivado_Lab/2023.2/settings64.sh 
vivado_lab
```

Follow the steps on https://fabric-testbed.atlassian.net/browse/FIP-1528?focusedCommentId=21607

1. Open Vivado > Quick Start > Open Hardware Manager

![Configuration Memory Device 1](./vivado_labtools/vivado-1.png)

2. Open target (see the green banner) and Auto Connect

![Configuration Memory Device 2](./vivado_labtools/vivado-2.png)

3. Add Configuration Memory Device

![Configuration Memory Device 3](./vivado_labtools/vivado-3.png)

4. Search: mt25qu01g-spi-x1_x2_x4 and select

![Configuration Memory Device 4](./vivado_labtools/vivado-4.png)
![Configuration Memory Device 5](./vivado_labtools/vivado-5.png)

5. Program Configuration Memory Device:
   - Select the bitstream and OK
   - Bitstream: <font color='red'>Use firesim.mcs</font>
   - There will be 2 steps (Step 1, 2) in the process.
   
![Configuration Memory Device 6](./vivado_labtools/vivado-6.png)
![Configuration Memory Device 7](./vivado_labtools/vivado-7.png)
![Configuration Memory Device 8](./vivado_labtools/vivado-8.png)
![Configuration Memory Device 9](./vivado_labtools/vivado-9.png)
![Configuration Memory Device 10](./vivado_labtools/vivado-10.png)
![Configuration Memory Device 11](./vivado_labtools/vivado-11.png)

6. Boot from configuration memory device

![Configuration Memory Device 12](./vivado_labtools/vivado-12.png)

7. Exit Vivado

![Configuration Memory Device 13](./vivado_labtools/vivado-13.png)



Execute PCIbus rescan on the worker node

```
/opt/mlnx-scripts/pcie_disable_fatal.sh 25:00.0
/opt/mlnx-scripts//pcie_disable_fatal.sh 25:00.1
echo 1 > /sys/bus/pci/devices/0000\:25\:00.0/remove
echo 1 > /sys/bus/pci/devices/0000\:25\:00.1/remove
echo 1 > /sys/bus/pci/rescan
```

#### <font color='red'>STOP - This is the end of flashing with Method-2</font> 

Rescan the PCI bus (This section is not used)

In [ ]:
#commands = list()
#commands.append(f'sudo bash -c "echo 1 > /sys/bus/pci/devices/0000\:1f\:00.0/remove"')
#commands.append(f'sudo lspci -vd 10ee:')
#commands.append(f'sudo bash -c "echo 1 > /sys/bus/pci/rescan"')
#commands.append(f'sudo lspci -vd 10ee:')

#for command in commands:
#    print(f'Executing {command}')
#    stdout, stderr = node.execute(command)

#print('Done')    

### Method 3 - Flash the card with xbflash2

#### 5.3.1 - Flash with xbflash2

xbflash2 - https://xilinx.github.io/XRT/master/html/xbflash2.html#


In [ ]:
commands = list()
commands.append(f"yes | sudo xbflash2 program --spi --image {bitfile_location}/{artifact} -d {fpga_bdf.replace('0000:','')}")
#commands.append(f"yes | sudo xbflash2 program --spi --image {bitfile_location}/{artifact_golden} -d {fpga_bdf.replace('0000:','')}")


for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)

print('Done')    

Execute PCIbus rescan on the worker node

```
/opt/mlnx-scripts/pcie_disable_fatal.sh 25:00.0
/opt/mlnx-scripts//pcie_disable_fatal.sh 25:00.1
echo 1 > /sys/bus/pci/devices/0000\:25\:00.0/remove
echo 1 > /sys/bus/pci/devices/0000\:25\:00.1/remove
echo 1 > /sys/bus/pci/rescan
```

#### <font color='red'>STOP - This is the end of flashing with Method-3</font> 

## Step 6 - Status and kernel drivers

### 6.1 - Use configDPDK to read the registers

In [ ]:
tools_location = '~/fpga-tools'

commands = list()
commands.append(f'cd {tools_location}/pcimem; make; [ ! -e {tools_location}/pcimem/configDPDK.sh ] && ln -s {tools_location}/configDPDK.sh')
commands.append(f'cd {tools_location}/pcimem && ./configDPDK.sh')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

### 6.2 - Install kernel module open-nic-driver

In [ ]:
tools_location = '~/fpga-tools'

commands = list()
commands.append(f'cd {tools_location}/open-nic-driver; make')
commands.append(f'sudo rmmod onic &> /dev/null')
commands.append(f'cd {tools_location}/open-nic-driver; sudo insmod onic.ko RS_FEC_ENABLED=0')
commands.append(f'lsmod | grep onic')

for command in commands:
   print(f'--- Node {node_name}: Executing command: {command}')
   stdout, stderr = node.execute(command)
print('--- Done')

After the process completes, shutoff the VM and __cold-reboot__ the worker node if you flashed persistent flash or __warm-reboot__ if flashing was into RAM. For the __cold-reboot__ case, it is more straightforward to delete the slice (Step 7), cold-reboot the worker node and re-create a slice with the flashed FPGA. 

## Step 7: Extend Slice

Get slice details and extend the slice. This cell is optional and can be executed as-needed.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show();

Renew by 14 days

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Step 8: Delete the slice

Delete the slice after completing the programming.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")